export cells as modules?



In [ ]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var moduleOutput = __dirname + '/.modules';
var importer = require('../Core');
var fs = require('fs');
var path = require('path');

function exportNotebook(notebook) {
    const name = path.basename(notebook).replace(/\.ipynb/ig, '');
    const parent = path.basename(path.dirname(notebook));
    if(!fs.existsSync(moduleOutput)) {
        fs.mkdirSync(moduleOutput);
    }
    if(!fs.existsSync(path.join(moduleOutput, parent))) {
        fs.mkdirSync(path.join(moduleOutput, parent));
    }
    if(!fs.existsSync(path.join(moduleOutput, parent, name))) {
        fs.mkdirSync(path.join(moduleOutput, parent, name));
    }
    
    return importer.getCells(notebook, ['*', 'code', 'markdown', 'raw'])
        .reduce((results, cell, i) => {
            var extension;
            if(cell.cell_type === 'markdown') {
                extension = '.md';
            } else if(cell.cell_type === 'markdown') {
                extension = '.txt';
            } else if(cell.language === 'javascript') {
                extension = '.js';
            } else if(cell.language === 'powershell') {
                extension = '.ps1';
            } else if(cell.language === 'csharp') {
                extension = '.cs';
            } else if(cell.language === 'python') {
                extension = '.py';
            } else if(cell.language === 'typescript') {
                extension = '.ts';
            } else if(cell.language === 'bash') {
                extension = '.sh';
            } else {
                throw 'unknown language or cell type: ' + notebook + ' (' + cell.cell_type + ',' + cell.language + ')';
            }
            const cellPath = path.join(moduleOutput, parent, name,
                                       'cell-' + i + extension);
            fs.writeFileSync(cellPath, cell.source.join(''));
            results.push(cellPath);
            return results;
        }, [])
}
module.exports = exportNotebook;


export all notebooks?

Export all notebooks to a structured folder where each cell has it's own file for linting?



In [ ]:
var glob = require('glob');
var path = require('path');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

function exportAll(project) {
    const notebooks = glob.sync('**/*.ipynb', {cwd: project});
    return notebooks.map(n => {
        return exportNotebook(path.join(project, n))
    })
}
module.exports = exportAll;


test notebook export?



In [ ]:
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var fs = require('fs');
var importer = require('../Core');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = path.resolve(__dirname + '/../');
var moduleOutput = PROJECT_PATH + '/Utilities/.modules';

var sortNumeric = (that) => {
    return that.sort((a, b) => parseInt(a.split(/[^0-9]/ig).join('')) - parseInt(b.split(/[^0-9]/ig).join('')));
}

describe('notebook export service', () => {
    beforeEach(() => {
        if(!fs.existsSync(moduleOutput)) {
            fs.mkdirSync(moduleOutput);
        }
        assert(fs.existsSync(PROJECT_PATH));
        assert(fs.existsSync(moduleOutput));
    })
    
    it('should export this file', () => {
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: moduleOutput});
        sortNumeric(files);
        assert(path.basename(files[0]) === 'cell-0.md');
        assert(path.basename(files[1]) === 'cell-1.js');
        assert(path.basename(files[2]) === 'cell-2.md');
    })
    
    it('should have comparable cells', () => {
        var importCells = importer.interpret(['notebook.ipynb[0]', 'notebook.ipynb[1]', 'notebook.ipynb[2]']);
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: moduleOutput});
        sortNumeric(files);
        var fsCell0 = fs.readFileSync(path.join(moduleOutput, files[0])).toString();
        var fsCell1 = fs.readFileSync(path.join(moduleOutput, files[1])).toString();
        var fsCell2 = fs.readFileSync(path.join(moduleOutput, files[2])).toString();
        assert(fsCell0 === importCells[0].markdown.join(''));
        assert(fsCell1 === importCells[0].fresher, fsCell1.length + ' != ' + importCells[0].fresher.length)
        assert(fsCell2 === importCells[1].markdown.join(''))
    })
    
    it('should export all notebooks', () => {
    //    var exported = exportAll(PROJECT_PATH);
    //    assert(exported.length > 0);
    })
})


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


In [ ]:
// TODO: delete .modules if everything checks out

// TODO: import tests


TODO:

import module as notebook cell?

import gist as notebook

import random instructions as notebook

rules:
- testing cells have zero markdown before them,
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable insolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'

